# [config.py](..\utils\config.py)

## import 导入链的注意事项

导入是将被导入模块中的名称添加到导入模块的命名空间中的过程。

假设当前模块名为 `M`，`from X import Xo` 会将 `Y` 直接添加到当前模块的命名空间：可以访问 `M.Xo`

而 `import X` 会将整个模块 `X` 作为一个对象添加到当前模块的命名空间：`M.X`（包括例如 `M.X.Xo`）等。

例子 1：

```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
from A import value, func  # 从A导入特定内容到B的命名空间

# 模块C (c.py)
import B

# 在C中可以使用:
print(B.value)  # 可以访问，输出100
print(B.func())  # 可以访问，输出"A.func"
```
例子 2：
```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
import A  # A成为B模块命名空间中的一个对象

# 模块C (c.py)
import B

# 在C中:
print(B.A.value)  # 可以访问，输出100
print(B.value)    # 错误! B中没有直接定义value
```
## 描述符
### 描述符协议

描述符协议规定了：`描述符` 作为某个属主类的 `类属性` 时，属主类使用该描述符时实际发生的行为。

一个类要成为 `描述符`，至少要实现下面三个特殊方法中的一个：

```python
__get__(self, instance, owner)
__set__(self, instance, value)
__delete__(self, instance)
```

其中 `instance` 是属主类实例，`owner` 是属主类本身。

```python
class Descriptor:

    def __init__(self):
        ...

    def __get__(self, instance, owner):
        ...

    def __set__(self, instance, value):
        ...

    def __delete__(self, instance):
        ...

class User:
    descriptor = Descriptor()

user = User()
```

当类似于 `= user.descriptor` 使用时，实际上是触发了 `descriptor.__get__(user, User)`。

`user.descriptor = 值` 实际上触发了 `descriptor.__set__(user, 值)`。

`del user.descriptor` 实际上触发了 `descriptor.__delete__(user)`。

### property 类

`property` 是一个实现了描述符协议的 Python 内置类，其实现：

```python
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc"

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    # 返回新的 Property 实例
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
```

### @property

使用方法如下

```python
class MyClass:
    def __init__(self):

    # value = property(函数 value(self))
    @property
    def value(self):
        ...

    # value = value.setter(函数 value(self, new_value))
    @value.setter
    def value(self, new_value):
        ...

    # value = value.setter(函数 value(self))
    @value.deleter
    def value(self):
        ...
```

所以相当于创建了三次 property 实例，最后该实例成为 MyClass 的一个类属性 value。

然后 MyClass 或其实例就可以按照 `描述符协议` 来使用 value。

## `__dict__` 和字典类的 `keys()`
### `__setattr__` 和 `__setitem__`
通过 `点表示法` 给实例的属性赋值时，会调用 `__setattr__`，默认情况下，会将该属性的值更新到该实例的 `__dict__`。

实现了 `__setitem__` 方法的类型，其实例使用 `[]` 索引法赋值时，会调用 `__setitem__`。
### `__dict__` 和字典类的 `keys()`

类型和实例都有 `__dict__`，其可以理解为一个字典。
实例的 `__dict__` 存储了实例属性（包括 `__init__` 中 `self.* =`为所有实例绑定的，以及动态绑定的），而类型的 `__dict__` 存储了类属性和方法。

字典 `dict` 有 `keys()`、`values()` 和 `items()` 方法。继承了字典的类自然也继承了这三个方法。
### Config 类的 `_as_attrs_`, `__setattr__`, `__setitem__`
注意到 `Config(PickleableDict, Documented)<——PickleableDict(Pickleable, dict)`，所以 `Config` 是字典类。

`Config` 作为字典类，以及 `_as_attrs_`, `_readonly_`, `_frozen_keys_`, `__setattr__`, `__setitem__` 的实现，构成了：
- `__dict__` 和 `dict` 是两个存储实例属性的结构；
- `_as_attrs_` 是 `false`：点赋值无效，只提供了索引赋值`dict`（且不能是`非强制更新 && _readonly_ || 非强制更新 && _frozen_keys_ && 索引的键在 dict 中` 的情况）；
- `_as_attrs_` 是 `true`：无论点赋值或索引赋值，`非强制更新 && _readonly_ || 非强制更新 && _frozen_keys_ && 索引的键在 dict 中` 决定了是否更新 `__dict__` 和 `dict`；

所以这种设计思想是：
- `_as_attrs_` 决定了点赋值是无效，还是被切入索引赋值；
- `非强制更新 && _readonly_ || 非强制更新 && _frozen_keys_ && 索引的键在 dict 中` 决定了是否更新 `dict`；
-  更新 `dict` 时，`_as_attrs_` 进一步决定了是否更新 `__dict__`；

# [checks.py](..\utils\checks.py)